# Pretraining a RoBERTa Model from Scratch



## Training a tokenizer and pretraining a transformer

Dans ce guide, nous allons entraîner un modèle de transformateur nommé KantaiBERT en utilisant les blocs de construction fournis par Hugging Face pour les modèles de type BERT. Nous avons couvert la théorie des éléments constitutifs du modèle que nous avons utilisé précédement.

Nous décrirons KantaiBERT en nous appuyant sur les connaissances acquises dans les guides précédents

KantaiBERT est un modèle de type Robustly Optimized BERT Pretraining Approach (RoBERTa) basé sur l'architecture de BERT. 


Les modèles BERT initiaux étaient sous-formés. RoBERTa augmente les performances des transformateurs de pré-apprentissage pour les tâches en aval. RoBERTa a amélioré la mécanique du processus de pré-formation. Par exemple, il n'utilise pas la tokenisation WordPiece mais descend à l'encodageByte Pair Encoding (BPE)

Dans ce guide, KantaiBERT, comme BERT, sera formé à l'aide de la modélisation du langage masqué. KantaiBERT sera formé comme un petit modèle avec 6 couches, 12 têtes et 84 095 008 paramètres. Il peut sembler que 84 millions de paramètres représentent un grand nombre de paramètres. 

Cependant, les paramètres sont répartis sur 6 couches et 12 têtes, ce qui le rend relativement petit. Un petit modèle rendra l'expérience de pré-entraînement fluide afin que chaque étape puisse être visualisée en temps réel sans attendre des heures pour voir un résultat


KantaiBERT est un modèle de type DistilBERT car il a la même architecture de 6 couches et 12 têtes. DistilBERT est une version distillée de BERT. Nous savons que les grands modèles offrent d'excellentes performances. Mais que faire si vous souhaitez exécuter un modèle sur un smartphone ? La miniaturisation a été la clé de l'évolution technologique. 

Les transformateurs devront suivre le même chemin lors de la mise en œuvre. L'approche Hugging Face utilisant une version distillée de BERT est donc un bon pas en avant. La distillation, ou d'autres méthodes de ce type à l'avenir, est un moyen intelligent de tirer le meilleur parti de la préformation et de la rendre efcace pour les besoins de nombreuses tâches en aval.

KantaiBERT implémentera un tokenizer byte-level byte-pair encoding comme celui utilisé par GPT-2. Les jetons spéciaux seront ceux utilisés par RoBERTa.


Il n'y a pas d'ID de type de jeton pour indiquer à quelle partie d'un segment un jeton fait partie. Les segments seront séparés avec le jeton de séparation \</s>.

KantaiBERT utilisera un ensemble de données personnalisé, formera un tokenizer, formera le modèle de transformateur, l'enregistrera et l'exécutera avec un exemple de modélisation de langage masqué.

Commençons à construire un transformateur de zéro.

## Étape 1 : Chargement de l'ensemble de données 

Les ensembles de données prêts à l'emploi offrent un moyen objectif d'entraîner et de comparer les transformateurs. 
 
 Le but de ce guide est de comprendre le processus d'apprentissage d'un transformateur avec des cellules de bloc-notes qui pourrait être exécuté en temps réel sans avoir à attendre des heures pour obtenir un résultat.

 
J'ai choisi d'utiliser les œuvres d'Emmanuel Kant (1724-1804), le philosophe allemand, qui fut l'incarnation du siècle des Lumières. L'idée est d'introduire une logique humaine et un raisonnement pré-entraîné pour les tâches de raisonnement en aval.


Project Gutenberg, https://www.gutenberg.org, propose une large gamme de livres électroniques gratuits qui peuvent être téléchargés au format texte. 

Vous pouvez utiliser d'autres livres si vous souhaitez créer vos propres ensembles de données personnalisés basés sur des livres. J'ai compilé les trois livres suivants d'Immanuel Kant dans un fichier texte nommé kant.txt :


* he Critique of Pure Reason
* The Critique of Practical Reason
* Fundamental Principles of the Metaphysic of Morals

kant.txt fournit un petit ensemble de données d'entraînement pour entraîner le modèle de transformateur de ce guide. 

Le résultat obtenu reste expérimental. Pour un projet réel.

L'ensemble de données est téléchargé automatiquement depuis GitHub :

vous pouvez utiliser curl pour le récupérer depuis GitHub

In [2]:
!curl -L https://raw.githubusercontent.com/PacktPublishing/Transformers-for-Natural-Language-Processing/master/Chapter03/kant.txt --output "kant.txt"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.7M  100 10.7M    0     0  20.5M      0 --:--:-- --:--:-- --:--:-- 20.4M


## Step 2: Installation Hugging Face transformersWe 

Nous devrons installer des transformateurs et des tokenizers Hugging Face, mais nous n'aurons pas besoin de TensorFlow dans cette instance de la VM Google Colab

In [1]:
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-wnwaxpi1
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-wnwaxpi1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 895 kB 5.3 MB/s 
     |████████████████████████████████| 3.3 MB 31.5 MB/s 
     |████████████████████████████████| 596 kB 44.3 MB/s 
     |████████████████████████████████| 59 kB 6.8 MB/s 
  Created wheel for transformers: filename=transformers-4.13.0.dev0-py3-none-any.whl size=3101540 sha256=c578e02210150d279d091b52e3928a5730f8ede84d27c5bae83a1d270793cf7d
  Stored in directory: /tmp/pip-ephem-wheel-cache-snvj8l71/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully unin

## Étape 3 : Formation d'un tokenizer

Dans cette section, le programme n'utilise pas de tokenizer pré-entraîné. Par exemple, un tokenizer GPT-2 pré-entraîné pourrait être utilisé. Cependant, le processus de formation de ce chapitre comprend la formation d'un tokenizer à partir de zéro. ByteLevelBPETokenizer() de Hugging Face sera formé à l'aide de kant.txt. 

Un tokenizer au niveau de l'octet divisera une chaîne ou un mot en une sous-chaîne ou un sous-mot. Il y a deux avantages principaux parmi tant d'autres

* Le tokenizer peut diviser les mots en composants minimaux. Ensuite, il fusionnera ces petits composants en d'autres statistiquement intéressants. Par exemple, "smaller" et smallest" peuvent devenir "small", "er" et "est".

Le tokenizer peut aller plus loin, et on pourrait obtenir « sm » et « all », par exemple. Dans tous les cas, les mots sont décomposés en jetons de sous-mots et en unités plus petites de parties de sous-mots telles que "sm" et "all" au lieu de simplement "small".

* Les morceaux de chaînes classés comme un jeton inconnu, utilisant l'encodage de niveau WorkPiece, disparaîtront pratiquement.


Dans ce modèle, nous allons entraîner le tokenizer avec les paramètres suivants :

* files=paths est le chemin d'accès à l'ensemble de données.
* vocab_size=52_000 est la taille de la longueur du modèle de notre tokenizer•
* min_fréquence=2 est le seuil de fréquence minimum.
* special_tokens=[] est une liste de jetons spéciaux


Dans ce cas, la liste des jetons spéciaux est :
* \<s> : un jeton de début
* \<pad> : un jeton de remplissage
* \</s> : un jeton de fin
* \<unk> : un jeton inconnu
* \<mask> : le jeton de masque pour la modélisation du langage


In [3]:
from pathlib import Path

from tokenizers import ByteLevelBPETokenizer

paths = [str(x) for x in Path(".").glob("**/*.txt")]

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])


## étape 4 : enregistrer les fichiers sur le disque Le tokenizer générera deux fichiers une fois entraînés : 
*  merges.txt, qui contient les sous-chaînes fusionnées 
* vocab.json, qui contient les index des sous-chaînes tokenisées 

Le programme crée d'abord le répertoire KantaiBERT puis enregistre les deux fichiers :

In [4]:
import os

token_dir = '/content/KantaiBERT'

if not os.path.exists(token_dir):
  os.makedirs(token_dir)

tokenizer.save_model('KantaiBERT')

['KantaiBERT/vocab.json', 'KantaiBERT/merges.txt']

## Étape 5 : Chargement des fichiers de tokenizer entraînés

Nous aurions pu charger des fichiers de tokenizer pré-entraînés. Cependant, nous avons formé notre propre tokenizer et sommes maintenant prêts à charger les fichiers :

In [5]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

tokenizer = ByteLevelBPETokenizer(    
    "./KantaiBERT/vocab.json",    
    "./KantaiBERT/merges.txt",
    )


Le tokenizer peut encoder une séquence

In [6]:
tokenizer.encode("The Critique of Pure Reason.").tokens

['The', 'ĠCritique', 'Ġof', 'ĠPure', 'ĠReason', '.']

On peut aussi demander à voir le nombre de jetons dans cette séquence :

In [7]:
tokenizer.encode("The Critique of Pure Reason.")

Encoding(num_tokens=6, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

Le tokenizer traite maintenant les jetons pour s'adapter à la variante du modèle BERT utilisée dans ce bloc-notes. Le post-processeur ajoutera un jeton de début et de fin, par exemple

In [8]:
tokenizer._tokenizer.post_processor = BertProcessing(    
    ("</s>", tokenizer.token_to_id("</s>")),    
    ("<s>", tokenizer.token_to_id("<s>")),
    )

In [9]:
tokenizer.enable_truncation(max_length=512)

Codons une séquence post-traitée

In [10]:
tokenizer.encode("The Critique of Pure Reason.")

Encoding(num_tokens=8, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])


Si nous voulons voir ce qui a été ajouté, nous pouvons demander au tokenizer d'encoder la séquence post-traitée en exécutant la cellule suivante

In [11]:
tokenizer.encode("The Critique of Pure Reason.").tokens

['<s>', 'The', 'ĠCritique', 'Ġof', 'ĠPure', 'ĠReason', '.', '</s>']

## Étape 6 : Vérification des contraintes de ressources : GPU et CUDA

KantaiBERT fonctionne à une vitesse optimale avec une unité de traitement graphique (GPU). Nous exécuterons d'abord une commande pour voir si une carte GPU NVIDIA est présente :

In [12]:
!nvidia-smi

Mon Nov  8 10:10:10 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [13]:
import torch

torch.cuda.is_available()

True

## Étape 7 : Définition de la configuration du modèle

Nous préformons un modèle de transformateur de type RoBERTa utilisant le même nombre de couches et de têtes qu'un transformateur DistilBERT. Le modèle aura une taille de vocabulaire définie sur 52 000, 12 têtes d'attention et 6 couches

In [14]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)



Nous allons explorer la configuration plus en détail à l'étape 9 : Initialiser un modèle à partir de zéro. Commençons par recréer le tokenizer dans notre modèle.

##  Étape 8 : Recharger le tokenizer dans les transformateurs

Nous sommes maintenant prêts à charger notre tokenizer formé, qui est notre tokenizer pré-entraîné dans RobertaTokenizer.from_pretained():


In [15]:
from transformers import RobertaTokenizer
 
tokenizer = RobertaTokenizer.from_pretrained( "./KantaiBERT", max_length=512)
 

file ./KantaiBERT/config.json not found


Maintenant que nous avons chargé notre tokenizer formé, initialisons un modèle RoBERTa à partir de zéro

## Étape 9 : Initialisation d'un modèle à partir de zéro

Dans cette section, nous allons initialiser un modèle à partir de zéro et examiner la taille du modèle. Le programme importe d'abord un modèle masqué RoBERTa pour la modélisation du langage.

In [16]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

## Étape 10 : Construire l'ensemble de données 

Le programme va maintenant charger l'ensemble de données ligne par ligne pour l'apprentissage par lots avec block_size=128 limitant la longueur d'un exemple :

In [17]:
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./kant.txt",
    block_size=128,
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


## Étape 11 : Définir un assembleur de données

Nous devons exécuter un assembleur de données avant d'initialiser le formateur.

 Un collecteur de données prélèvera des échantillons de l'ensemble de données et les rassemblera en lots. 
 
 Les résultats sont des objets de type dictionnaire. Nous préparons un processus d'échantillonnage par lots pour la modélisation du langage masqué (MLM) en définissant mlm=True.
 
 Nous avons également défini le nombre de jetons masqués pour entraîner mlm_probability=0.15. 
 
 Cela déterminera le pourcentage de jetons masqués pendant le processus de pré-entraînement. Nous initialisons maintenant data_collator avec notre tokenizer, MLM activé et la proportion de jetons masqués définie sur 0,15

In [18]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

## Étape 12 : Initialisation du formateur 

Les étapes précédentes ont préparé les informations nécessaires à l'initialisation du formateur. 

L'ensemble de données a été tokenisé et chargé. Notre modèle est construit. Le collecteur de données a été créé. Le programme peut maintenant initialiser le formateur. À des fins éducatives, le programme entraîne le modèle rapidement. Le nombre d'époques est limité à un. Le GPU est pratique car nous pouvons partager les lots et multi-traiter les tâches de formation

In [19]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./EsperBERTo",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

## Étape 13 : Préformation du modèle 

Tout est prêt. Le formateur est lancé avec une ligne de code :

In [20]:
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
***** Running training *****
  Num examples = 170964
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 2672
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss
500,6.606600
1000,5.711500
1500,5.226400
2000,4.978500
2500,4.824400




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=2672, training_loss=5.425474223976364, metrics={'train_runtime': 1178.5633, 'train_samples_per_second': 145.061, 'train_steps_per_second': 2.267, 'total_flos': 873620128952064.0, 'train_loss': 5.425474223976364, 'epoch': 1.0})

## Étape 14 : Sauvegarder le modèle final (+tokenizer + config) sur le disque

Nous allons maintenant sauvegarder le modèle et la configuration

In [21]:
trainer.save_model("./KantaiBERT")

Saving model checkpoint to ./KantaiBERT
Configuration saved in ./KantaiBERT/config.json
Model weights saved in ./KantaiBERT/pytorch_model.bin


## Étape 15 : Modélisation du langage avec FillMaskPipeline

Nous allons maintenant importer une tâche de masque de remplissage de modélisation du langage. Nous utiliserons notre modèle entraîné et notre tokenizer entraîné pour effectuer une modélisation de langage masqué

In [22]:
from transformers import pipeline

fill_mask = pipeline(    
    "fill-mask",    
    model="./KantaiBERT",    
    tokenizer="./KantaiBERT"
    )

loading configuration file ./KantaiBERT/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.13.0.dev0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 52000
}

loading configuration file ./KantaiBERT/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dr

In [23]:
fill_mask("Human thinking involves human <mask>.")

[{'score': 0.04041118547320366,
  'sequence': 'Human thinking involves human reason.',
  'token': 393,
  'token_str': ' reason'},
 {'score': 0.014237454161047935,
  'sequence': 'Human thinking involves human experience.',
  'token': 531,
  'token_str': ' experience'},
 {'score': 0.009888945147395134,
  'sequence': 'Human thinking involves human conceptions.',
  'token': 605,
  'token_str': ' conceptions'},
 {'score': 0.009337211959064007,
  'sequence': 'Human thinking involves human it.',
  'token': 306,
  'token_str': ' it'},
 {'score': 0.007209516130387783,
  'sequence': 'Human thinking involves human time.',
  'token': 526,
  'token_str': ' time'}]